In [16]:
import pandas as pd
import numpy as np

dms_scores = pd.read_csv("../data/DMS/antibody/dms_antibodies_XBB15_JN1_agg.csv")

# We will use "site scores" as antibody features
# site_mat should be normalized by sum = 1 for each antibody
site_mat = dms_scores.groupby(['antibody','site']).sum().reset_index().assign(
    mut_escape = lambda x: x.groupby('antibody')['mut_escape'].transform(lambda y: y/y.sum())
)

use_abs = set(pd.unique(site_mat['antibody']))
print("Number of mAbs:", len(use_abs))

# To integrate the results from JN.1 and XBB.1.5 DMS, scores on the key deletion V483del is removed during the analysis
exclude_sites = [483]

# selected significant sites as features
# (normalized site score > 0.1 for at least 0.2% mAbs in the dataset)
use_sites = site_mat.query("mut_escape > 0.1").groupby('site')['antibody'].count()
use_sites = use_sites[use_sites >= 0.002*len(use_abs)].index.to_list()
print("Number of significant sites:", len(np.setdiff1d(use_sites, exclude_sites)))

site_mat = site_mat.query('site in @use_sites and site not in @exclude_sites')
# normalize again
site_mat = site_mat.assign(
    value = lambda x: x.groupby(
    ['antibody'])['mut_escape'].transform(lambda x: x/x.sum())
).pivot(columns='site',values="mut_escape",index="antibody").fillna(0.0)
site_mat.index.name = 'id'
site_mat.to_csv("../data/DMS/antibody/_site_mat.csv")

Number of mAbs: 2688
Number of significant sites: 71


In [17]:
# Load some useful information of the mAbs
mAb_info = pd.read_csv("../data/_mAb_info_clean.csv").query("id in @use_abs")
print(len(mAb_info), len(use_abs))

2688 2688


In [18]:
# statistics
XBB15_mAbs = pd.unique(dms_scores.query('antigen == "XBB.1.5_RBD"')['antibody'])
JN1_mAbs = pd.unique(dms_scores.query('antigen == "JN.1_RBD"')['antibody'])
mAbs_both = np.intersect1d(XBB15_mAbs, JN1_mAbs)
print(f"XBB.1.5 library DMS mAbs: {len(XBB15_mAbs)}")
print(f"JN.1 library DMS mAbs: {len(JN1_mAbs)}")
print(f"Assayed in both libraries: {len(mAbs_both)}")

print("XBB.1.5 DMS mAbs by source:")
print(mAb_info[['id','source']].query('id in @XBB15_mAbs').groupby('source').count())

print("JN.1 DMS mAbs by source:")
print(mAb_info[['id','source']].query('id in @JN1_mAbs').groupby('source').count())

XBB.1.5 library DMS mAbs: 2286
JN.1 library DMS mAbs: 766
Assayed in both libraries: 364
XBB.1.5 DMS mAbs by source:
                                 id
source                             
BA.1 + BA.5/BF.7 infection       97
BA.1 BTI                         52
BA.1 BTI + BA.5/BF.7 infection  161
BA.2 + BA.5/BF.7 infection      109
BA.2 BTI                         57
BA.2 BTI + BA.5/BF.7 infection  135
BA.5 + JN.1 infection            25
BA.5 + XBB infection            293
BA.5 BTI                         78
BA.5 BTI + HK.3 infection       274
BA.5 BTI + JN.1 infection        58
BA.5 BTI + XBB infection        297
BF.7 BTI                         53
SARS                              1
SARS exposure                     6
WT convalescents                 10
WT-phage display                  1
XBB BTI                         264
XBB infection                   270
long-term BA.1 BTI               16
long-term BA.5 BTI               23
long-term BA.5 infection          6
JN.1 DMS mAbs by so

In [19]:
from itertools import combinations
from scipy.spatial.distance import jensenshannon
from multiprocessing import Pool
def calc_sqrt_JSD(pair):
    array, idx = pair
    return ([idx, [
        np.sqrt(jensenshannon(array[idx, :], array[j, :])) for j in range(array.shape[0])
    ]])

site_mat_arr = site_mat.to_numpy()
n_threads = 32
with Pool(n_threads) as pool:
    divergences = pool.map(calc_sqrt_JSD, [[site_mat_arr, x] for x in range(len(site_mat.index))])
    
dist = np.empty((len(site_mat), len(site_mat)))
for i, x in divergences:
    dist[i,:] = x

dist = pd.DataFrame(dist, index=site_mat.index.to_list(), columns=site_mat.index.to_list())


In [20]:
# define a Graph for leiden clustering
import leidenalg as la
import igraph as ig

class DMSProfileGraph(ig.Graph):
    def __init__(self, edges_df):
        nodes = pd.unique(edges_df['index'])
        self.sample2node = {}
        self.node2sample = {}
        edges_node = set()
        
        for i in range(len(nodes)):
            self.node2sample[i] = nodes[i]
            self.sample2node[nodes[i]] = i
        
        for u, v, w in edges_df.to_numpy():
            _u = self.sample2node[u]
            _v = self.sample2node[v]
            edges_node.add((min(_u,_v), max(_u,_v)))
            
        super().__init__(n = len(nodes), edges = edges_node)
        
    def __len__(self):
        return len(self.sample2node)

import logomaker
from matplotlib import rcParams
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

def site_to_pos(sites, split='+'):
    sites = sorted(sites, key=lambda x: [int(y) for y in x.split(split)])
    site2pos = {}
    for i in range(len(sites)):
        site2pos[sites[i]] = i
    
    return sites, site2pos

def plot_res_logo(res, prefix, by='name', site_thres=0.1, width=26, shownames={}, num_per_page = 10, force_plot_sites = None, force_ylim = None, highlight_res = {}):
    rcParams['pdf.fonttype'] = 42

    res["site"] = res["site"].astype(str)
    flat_res = res.rename(columns={by:'antibody'}).pivot(index=['antibody', 'site'], columns='mutation', values='mut_escape').fillna(0)
    sites_total_score = flat_res.sum(axis=1)

    strong_sites = list(pd.unique(sites_total_score[sites_total_score > site_thres].reset_index()['site']))
    plot_sites = strong_sites
    
    if force_plot_sites is not None:
        plot_sites = force_plot_sites
    
    flat_res = flat_res.query('site in @plot_sites')
    Abs = flat_res.index.get_level_values('antibody').unique()
    Npages = len(Abs) // num_per_page + 1
    
    plot_sites, site2pos = site_to_pos(plot_sites)
    
    with PdfPages(prefix+'_aa_logo.pdf') as pdf:
        for p in range(Npages):
            Abs_p = Abs[p*10:min(len(Abs),(p+1)*10)]
            fig = plt.figure(figsize=(width,len(Abs_p)*4.6)).subplots_adjust(wspace=0.2,hspace=0.5)

            for i in range(len(Abs_p)):
                ab = Abs_p[i]
                _ = flat_res.loc[ab, :]
                add_sites = np.setdiff1d(plot_sites, _.index)
                for _site in add_sites:
                    _.loc[_site,:] = 0.0
                _.index = [site2pos[i] for i in _.index]
                ax = plt.subplot(len(Abs_p), 1, i+1)
                logo = logomaker.Logo(_,
                               ax=ax, 
                               color_scheme='dmslogo_funcgroup', 
                               vpad=.1, 
                               width=.8)
                logo.style_xticks(anchor=0, spacing=1, rotation=90, fontsize=16)
                _max = np.sum(_.to_numpy(), axis=1).max()
                ax.yaxis.set_tick_params(labelsize=20)
                if force_ylim is not None:
                    ax.set_ylim(0, force_ylim)
                elif _max < 3:
                    ax.set_ylim(0,3)
                    ax.set_yticks(range(0, 3, 1))
                elif _max < 5:
                    ax.set_yticks(range(0, int(_max)+1, 1))
                elif _max < 8:
                    ax.set_yticks(range(0, int(_max)+1, 2))
                else:
                    ax.set_yticks(range(0, int(_max)+1, 3))

                for color, sites in highlight_res.items():
                    if ifsite in plot_sites:
                        logo.highlight_position(p=site2pos[ifsite], color=color, alpha=.2)

                ax.set_xticklabels(plot_sites)

                if ab in shownames:
                    ax.set_title(shownames[ab], fontsize=24, fontweight="bold")
                else:
                    ax.set_title(ab, fontsize=24, fontweight="bold")
            pdf.savefig()
            plt.close()


In [21]:
import umap.umap_ as umap

n_neighbors = 12
seed = 2024
umap_params = {'n_neighbors': n_neighbors, 'min_dist':0.8, 'metric':'precomputed', 'random_state': seed}
la_params = {'partition_type':la.RBConfigurationVertexPartition, 'resolution_parameter':2.0, 'seed': seed}

edges = (
    dist.reset_index().melt(id_vars="index")
        .query('index != variable').sort_values('value',ascending=True)
        .groupby('index').head(n_neighbors)
        .reset_index(drop=True)
)
knn_graph = DMSProfileGraph(edges_df = edges)

partition = la.find_partition(knn_graph, **la_params).membership
node_id = [knn_graph.sample2node[i] for i in dist.index]

umap = pd.DataFrame(umap.UMAP(**umap_params).fit_transform(dist), index=site_mat.index, columns=['UMAP1', 'UMAP2']).assign(
    cluster = ['C%02d'%partition[i] for i in node_id])

# If the mAb_info contains a column as cluster annotation references

# _ref = pd.read_csv("../data/DMS/antibody/_clustering.csv")
# umap = umap.merge(
#     _ref[['id','new_group']], left_index=True, right_on='id', how='left')
# contigmat = umap.groupby(['cluster','new_group'])['id'].count().reset_index().pivot(index='new_group', columns='cluster', values='id').fillna(0).astype(int)
# contigmat.to_csv('../data/DMS/antibody/_group_vs_clusters.csv')
# cluster_rename = contigmat.idxmax(axis=0).to_dict()

# or annotate the clusters manually
cluster_rename = {'C00': 'F3',
 'C01': 'F1.2',
 'C02': 'F3',
 'C03': 'A1',
 'C04': 'D3',
 'C05': 'A2',
 'C06': 'E3',
 'C07': 'F3',
 'C08': 'B',
 'C09': 'A1',
 'C10': 'E3',
 'C11': 'E1/E2.1',
 'C12': 'E2.2',
 'C13': 'F1.1',
 'C14': 'D4',
 'C15': 'B',
 'C16': 'E3',
 'C17': 'D3',
 'C18': 'D3',
 'C19': 'F3',
 'C20': 'E3',
 'C21': 'D2'}

src_groups = {
    'XBB infection': "XBB",
    'XBB BTI': "XBB",
    'BA.5 + XBB infection': "XBB",
    'BA.5 BTI + HK.3 infection': "XBB", 
    'BA.5 BTI + XBB infection': "XBB",
    'BA.5 + JN.1 infection': "JN.1",
    'BA.5 BTI + JN.1 infection': "JN.1",
}

umap = umap.merge(
    mAb_info[['id', 'source','paper_reactivity','D614G_IC50', 
          'BA1_IC50', 'BA2_IC50','BA5_IC50', 'XBB1_5_IC50', 'HK3_1_IC50', 'JN1_IC50', 
          'JN1_F456L_IC50',"JN1_R346T_F456L_IC50","JN1_F456L_A475V_IC50","KP3_IC50", "KP3_A475V_IC50",
          'v_gene_H', 'd_gene_H', 'j_gene_H', 'v_gene_L', 'j_gene_L', 'v_domain_shm_ratio_H','v_domain_shm_ratio_L','ACE2_competition'
         ]],
    left_on='id', right_on='id', how='outer',
).assign(
    new_group=lambda x:[cluster_rename[y] for y in x['cluster']],
    XBB15_dms_assayed = lambda x:[(y in XBB15_mAbs) for y in x['id']],
    JN1_dms_assayed = lambda x:[(y in JN1_mAbs) for y in x['id']],
    # is_involved = lambda x:[(src in src_groups) for src in x['source']]
    # src_group=lambda x:[src_groups[y] if y in src_groups else "others" for y in x['source']]
)
umap.set_index('id').to_csv("../data/DMS/antibody/_clustering.csv")


/gshare/xielab/jianfc/miniforge3/envs/antibody/lib/python3.10/site-packages/umap/umap_.py:1857: UserWarning: using precomputed metric; inverse_transform will be unavailable
  warn("using precomputed metric; inverse_transform will be unavailable")
/gshare/xielab/jianfc/miniforge3/envs/antibody/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [22]:
df = umap.query('source in @src_groups')

cnt = df.groupby("source")['id'].count().to_dict()
df.assign(src_cnt = lambda x: [cnt[y] for y in x['source']]).query('src_cnt > 100').assign(
    contribution = lambda x: 1.0/x['src_cnt'],
).groupby(["source", "new_group"])['contribution'].agg(['sum', 'count']).reset_index().assign(
    label = lambda x: [y+' ('+str(round(cnt[y]))+')' for y in x['source']]).to_csv("../data/DMS/antibody/_group_stat.csv", index=None)

def ic50_weighting(x, low=0.001, high=1):
    if pd.isna(x) or x > high:
        return 0
    if x < low:
        return 1
    res = -np.log10(x)
    res_l = -np.log10(low)
    res_h = -np.log10(high)
    return 1.0-(res-res_l)/(res_h-res_l)


neut_weighted_calc = []

for use_neut in ['BA5_IC50', 'JN1_IC50', 'XBB1_5_IC50', 'HK3_1_IC50', 'JN1_R346T_F456L_IC50', 'KP3_IC50']:
    datax = df.assign(
        neut_weight = lambda x: [ic50_weighting(y) for y in x[use_neut]]
    )

    cnt = datax.groupby("source")['neut_weight'].sum().to_dict()
    cnt_orig = datax.groupby("source")['neut_weight'].count().to_dict()
    x = datax[['source', use_neut,'neut_weight', 'new_group']].groupby(['source', 'new_group'])['neut_weight'].sum().reset_index().assign(
        cnt=lambda x: [cnt[y] for y in x['source']], cnt_orig=lambda x: [cnt_orig[y] for y in x['source']])
    x['percentage'] = x['neut_weight'] * 100 / x['cnt']
    x['effectiveness'] = x['neut_weight'] / x['cnt_orig']
    
    neut_weighted_calc.append(x.assign(
        weight = use_neut, use_mAbs = 'all'
    ))
    
    # split cross and specific
    for spec in ['cross', 'specific']:
        datax = df.query('paper_reactivity == @spec').assign(
            neut_weight = lambda x: [ic50_weighting(y) for y in x[use_neut]]
        )

        cnt = datax.groupby("source")['neut_weight'].sum().to_dict()
        cnt_orig = datax.groupby("source")['neut_weight'].count().to_dict()
        x = datax[['source', use_neut,'neut_weight', 'new_group']].groupby(['source', 'new_group'])['neut_weight'].sum().reset_index().assign(
            cnt=lambda x: [cnt[y] for y in x['source']], cnt_orig=lambda x: [cnt_orig[y] for y in x['source']])
        x['percentage'] = x['neut_weight'] * 100 / x['cnt']
        x['effectiveness'] = x['neut_weight'] / x['cnt_orig']

        neut_weighted_calc.append(x.assign(
            weight = use_neut, use_mAbs = spec
        ))
pd.concat(neut_weighted_calc).to_csv(f"../data/DMS/antibody/_neut_weight_group_stat.csv", index=None)


In [23]:

_cnt = dms_scores[["antibody", "antigen"]].drop_duplicates().groupby(["antibody"])['antigen'].count().reset_index().rename(columns={'antigen':'count'})

dms_ag_avg = dms_scores.groupby(["antibody", "site", "mutation"])['mut_escape'].sum().reset_index().merge(
        _cnt, on="antibody").assign(mut_escape = lambda x: x['mut_escape'] / x['count']).drop(columns=['count'])

dms_ag_avg.to_csv("../data/DMS/antibody/_XBB15_JN1_aggregate_dms_scores.csv", index=None)

dms_ag_avg = dms_ag_avg.merge(
        umap[['new_group', 'source', 'cluster', 'id']], 
    left_on='antibody', right_on='id',how='left'
).drop(columns=['id']).dropna()



In [24]:
dms_scores_annotated = dms_scores.merge(umap[['id','source', 'new_group', 'paper_reactivity']], how='left', left_on='antibody', right_on='id')

In [10]:
# generate some logo plots using aggreagted XBB.1.5-JN.1 DMS scores
cluster_cnt = umap.groupby("cluster")['id'].count().to_dict()
group_cnt = umap.groupby("new_group")['id'].count().to_dict()

cluster_mean = dms_ag_avg.groupby(
    ["site","mutation","cluster"])['mut_escape'].sum().reset_index().assign(
    mut_escape = lambda x: [x.loc[i, 'mut_escape']/cluster_cnt[x.loc[i, 'cluster']] for i in x.index]
)
group_mean = dms_ag_avg.groupby(
    ["site","mutation","new_group"])['mut_escape'].sum().reset_index().assign(
    mut_escape = lambda x: [x.loc[i, 'mut_escape']/group_cnt[x.loc[i, 'new_group']] for i in x.index]
)

group_mean.to_csv("../data/DMS/antibody/_scores_group_mean.csv", index=None)
cluster_mean.to_csv("../data/DMS/antibody/_scores_cluster_mean.csv", index=None)

group_mean.groupby(['new_group','site'])['mut_escape'].sum().reset_index().to_csv("../data/DMS/antibody/_site_scores_group_sum.csv", index=None)

plot_res_logo(group_mean.query('mut_escape > 0.03'), prefix="../plots/Extended/DMS/antibody/DMS_group_mean", by="new_group", site_thres=1, width=24)
plot_res_logo(cluster_mean.query('mut_escape > 0.03'), prefix="../plots/Extended/DMS/antibody/DMS_cluster_mean", by="cluster", site_thres=1, width=24)


/tmp/ipykernel_41454/2945337053.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res["site"] = res["site"].astype(str)
/tmp/ipykernel_41454/2945337053.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res["site"] = res["site"].astype(str)


In [11]:
for g_group, y in [
    (['A1', 'A2', 'F3', 'B'], 8),
    (['D2', 'D3', 'D4'], 7),
    (['E1/E2.1', 'E2.2', 'E3','F1.1', 'F1.2'], 10),
]:
    plot_res_logo(group_mean.query('mut_escape > 0.03 and new_group in @g_group'), 
                  prefix=f"../plots/Extended/DMS/antibody/DMS_group_{'-'.join(g_group).replace('/','_')}_mean", by="new_group", site_thres=2, width=y)


/tmp/ipykernel_41454/2945337053.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res["site"] = res["site"].astype(str)
/tmp/ipykernel_41454/2945337053.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res["site"] = res["site"].astype(str)
/tmp/ipykernel_41454/2945337053.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [12]:
# Split by epitope group. Average by source
def calc_mAbs_by_source(res, group, src, group_col="new_group", src_col="source"):
    _use_res = res.query(f"{src_col} in @src and {group_col} in @group")
    _cnt = _use_res[["antibody", src_col, group_col]].drop_duplicates().groupby([src_col, group_col])['antibody'].count().reset_index().rename(columns={'antibody':'count'})
    return _use_res.groupby([src_col, group_col, "site", "mutation"])['mut_escape'].sum().reset_index().merge(
        _cnt, on=[src_col, group_col]).assign(**{
        'mut_escape': lambda x: x['mut_escape'] / x['count'],
        src_col: lambda x: x[src_col]+' ('+x['count'].astype(str)+')'
    })
    return _use_res.query('mut_escape > 0.01')

def plot_wrapper(df, g, use_src, file, src, out_dir = None):
    df_avg = calc_mAbs_by_source(df, g, use_src, src_col=src)
    if out_dir is not None:
        os.makedirs(out_dir, exist_ok=True)
        df_avg.to_csv(os.path.join(out_dir, f"{g.replace('/','_')}_{src}.csv"), index=None)
    plot_res_logo(df_avg, file, src, 0.5, 24)

show_groups = pd.unique(umap['new_group'])
show_src = list(src_groups.keys())
# show_src = ['BA.5 BTI + XBB infection', 'BA.5 BTI + HK.3 infection', 'BA.5 BTI + JN.1 infection']


In [13]:
import os
os.makedirs("../plots/Extended/DMS/antibody/group_by_source", exist_ok=True)

with Pool(len(show_groups)) as pool:
    pool.starmap(
        plot_wrapper, [[
            dms_ag_avg, g, show_src, 
            f"../plots/Extended/DMS/antibody/group_by_source/Group_mean_{g.replace('/','_')}_by_source", "source"
        ] for g in show_groups]
    )


In [14]:
with Pool(len(show_groups)) as pool:
    pool.starmap(
        plot_wrapper, [[
            dms_scores_annotated.query("antigen == 'XBB.1.5_RBD'"), g, show_src, 
            f"../plots/Extended/DMS/antibody/group_by_source_XBB15/Group_mean_{g.replace('/','_')}_by_source", "source"
        ] for g in show_groups]
    )


In [15]:
# Split by epitope group. Average by assayed antigen. only include mAbs that were assayed in both antigens

os.makedirs("../plots/Extended/DMS/antibody/group_by_antigen", exist_ok=True)

_ = dms_scores_annotated.query("antibody in @mAbs_both")
with Pool(len(show_groups)) as pool:
    pool.starmap(
        plot_wrapper, [[
            _, g, ["XBB.1.5_RBD", "JN.1_RBD"], 
            f"../plots/Extended/DMS/antibody/group_by_antigen/Group_mean_{g.replace('/','_')}_by_antigen", "antigen", "../data/DMS/antibody/both_mAbs_grouped"
        ] for g in show_groups]
    )


In [15]:
# generate the logo plot for each mAb
# this is time-consuming

groups = pd.unique(umap['new_group'])
with Pool(len(groups)) as pool:
    pool.starmap(plot_res_logo, [[
        dms_scores_annotated.assign(
            label = lambda x: x['antibody']+'('+x['antigen']+')',
        ).query('new_group == @group and mut_escape > 0.01')[['site','mutation','mut_escape','label']], 
        f"../plots/Extended/DMS/antibody/mAbs_dms_logo/mAb_dms_{group.replace('/','_')}", 
        "label", 
        0.1, 
        30,
    ] for group in groups])